In [1]:
# This notebook will be used to run the model in a way that makes it easier to develop new components of the model
import nasem_dairy as nd
import pandas as pd
import math
from icecream import ic


def run_dev_model():
########################################
# Step 1: Read User Input
########################################
    # animal_input is a dictionary with all animal specific parameters
    # diet_info is a dataframe with the user entered feed ingredients and %DM intakes
    # diet_info, animal_input, equation_selection = nd.read_csv_input('./input.csv')
    diet_info, animal_input, equation_selection = nd.read_csv_input('./input_drycow.csv')

    # list_of_feeds is used to query the database and retrieve the ingredient composition, stored in feed_data
    list_of_feeds = diet_info['Feedstuff'].tolist()
    feed_data = nd.fl_get_rows(list_of_feeds, '../../src/nasem_dairy/data/diet_database.db')

    # Set feed inclusion percentages
    diet_info['Fd_DMInp'] = diet_info['kg_user'] / diet_info['kg_user'].sum()

    # Calculate additional physiology values
    animal_input['An_PrePartDay'] = animal_input['An_GestDay'] - animal_input['An_GestLength']
    animal_input['An_PrePartWk'] = animal_input['An_PrePartDay'] / 7

########################################
# Step 2: DMI Equations
########################################
    # Need to precalculate Dt_NDF for DMI predicitons, this will be based on the user entered DMI (animal_input['DMI])
    Dt_NDF = nd.NDF_precalculation(diet_info, feed_data)

    if equation_selection['DMIn_eqn'] == 8: 
        animal_input['DMI'] = nd.calculate_Dt_DMIn_Lact1(animal_input['An_Parity_rl'], animal_input['Trg_MilkProd'], animal_input['An_BW'], animal_input['An_BCS'],
                                                         animal_input['An_LactDay'], animal_input['Trg_MilkFatp'], animal_input['Trg_MilkTPp'], animal_input['Trg_MilkLacp'])
    
    # Predict DMI for heifers
    if equation_selection['DMIn_eqn'] in [2,3,4,5,6,7,12,13,14,15,16,17]:
        animal_input['DMI'] = nd.heifer_growth(equation_selection['DMIn_eqn'], diet_info.loc['Diet', 'Fd_NDF'], animal_input['An_BW'], animal_input['An_BW_mature'], animal_input['An_PrePartWk'], nd.coeff_dict)

    # Predict DMI for dry cows
    if equation_selection['DMIn_eqn'] in [10,11]:
        animal_input['DMI'] = nd.dry_cow_equations(equation_selection['DMIn_eqn'], animal_input['An_BW'], animal_input['An_PrePartWk'], animal_input['An_GestDay'], animal_input['An_GestLength'], Dt_NDF, nd.coeff_dict)

    del(Dt_NDF)
########################################
# Step 3: Feed Based Calculations
########################################
    diet_info = nd.get_nutrient_intakes(diet_info, feed_data, animal_input['DMI'], equation_selection, nd.coeff_dict)

########################################
# Step 4: Micronutrient Calculations
########################################
    df_minerals, mineral_values = nd.mineral_intakes(animal_input['An_StatePhys'], feed_data, diet_info)

    df_vitamins = nd.vitamin_supply(feed_data, diet_info)
    
########################################
# Step 5: Microbial Protein Calculations
########################################
    Du_MiN_NRC2021_g, Rum_DigNDFIn, Rum_DigStIn = nd.calculate_Du_MiN_g(diet_info.loc['Diet', 'Fd_NDFIn'], animal_input['DMI'], diet_info.loc['Diet', 'Fd_St_kg/d'], diet_info.loc['Diet', 'Fd_CP_kg/d'], diet_info.loc['Diet', 'Fd_ADF_kg/d'], 
                                    diet_info.loc['Diet', 'Fd_ForWetIn'], diet_info.loc['Diet', 'Fd_RUPIn'], diet_info.loc['Diet', 'Fd_ForNDFIn'], diet_info.loc['Diet', 'Dt_RDPIn'], nd.coeff_dict)    
    
########################################
# Step 6: Amino Acid Calculations
########################################
    AA_values, Du_MiCP_g = nd.AA_calculations(Du_MiN_NRC2021_g, feed_data, diet_info, animal_input, nd.coeff_dict)

########################################
# Step 7: Other Calculations
########################################
# Intake calculations that require additional steps, need results from other calculations or values that need to be calculated for other functions

    GrUter_BWgain, GrUter_Wt = nd.calculate_GrUter_BWgain(animal_input['Fet_BWbrth'], animal_input['An_AgeDay'], animal_input['An_GestDay'], animal_input['An_GestLength'], animal_input['An_LactDay'], animal_input['An_Parity_rl'], nd.coeff_dict)

    # This function could be renamed as it is doing all the DE intake calculations
    An_DEIn, An_DENPNCPIn, An_DETPIn, An_DigNDFIn, An_DEStIn, An_DEFAIn, An_DErOMIn, An_DENDFIn, Fe_CP, Fe_CPend_g, Du_idMiCP_g = nd.calculate_An_DEIn(diet_info.loc['Diet', 'Fd_DigNDFIn_Base'], diet_info.loc['Diet', 'Fd_NDFIn'], 
                                                                                                                    diet_info.loc['Diet', 'Fd_DigStIn_Base'], diet_info.loc['Diet', 'Fd_St_kg/d'], 
                                                                                                                    diet_info.loc['Diet','Fd_DigrOMtIn'], diet_info.loc['Diet', 'Fd_CPIn'], diet_info.loc['Diet', 'Fd_RUPIn'],
                                                                                                                    diet_info.loc['Diet', 'Fd_idRUPIn'], diet_info.loc['Diet', 'Fd_NPNCPIn'], diet_info.loc['Diet', 'Fd_DigFAIn'], 
                                                                                                                    Du_MiCP_g, animal_input['An_BW'], animal_input['DMI'], nd.coeff_dict)

    # Predicted milk protein
    Mlk_NP_g, An_DigNDF, An_MPIn, An_DEInp = nd.calculate_Mlk_NP_g(AA_values, diet_info.loc['Diet', 'Fd_idRUPIn'], Du_idMiCP_g, An_DEIn, An_DETPIn, An_DENPNCPIn, An_DigNDFIn, An_DEStIn, An_DEFAIn, An_DErOMIn, An_DENDFIn, animal_input['An_BW'], animal_input['DMI'], animal_input['An_StatePhys'], nd.coeff_dict)

    # Net energy/Metabolizable energy
    An_NE, An_NE_In, An_MEIn, Frm_NPgain = nd.calculate_An_NE(diet_info.loc['Diet', 'Fd_CPIn'], diet_info.loc['Diet', 'Fd_FAIn'], Mlk_NP_g, An_DEIn, An_DigNDF, Fe_CP, Fe_CPend_g, animal_input['DMI'], animal_input['An_BW'], animal_input['An_BW_mature'], 
                                     animal_input['Trg_FrmGain'], animal_input['Trg_RsrvGain'], GrUter_BWgain, nd.coeff_dict)

########################################
# Step 8: Requirement Calculations
########################################
    # Metabolizable Energy Requirements
    Trg_MEuse, An_MEmUse, An_MEgain, Gest_MEuse, Trg_Mlk_MEout, Trg_NEmilk_Milk, Frm_NEgain, Rsrv_NEgain = nd.calculate_ME_requirement(animal_input['An_BW'], animal_input['DMI'], animal_input['Trg_MilkProd'], animal_input['An_BW_mature'], animal_input['Trg_FrmGain'],
                                                                                          animal_input['Trg_MilkFatp'], animal_input['Trg_MilkTPp'], animal_input['Trg_MilkLacp'], animal_input['Trg_RsrvGain'], GrUter_BWgain, nd.coeff_dict)
    
    # Calculate some values for the heifer adjustment to MP requirement, this will be changed in the future and is placed here to avoid cluttering the calculate_MP_requirement function  
    An_DigTPaIn = nd.temp_calc_An_DigTPaIn(Fe_CP, diet_info)
    An_GasEOut = nd.temp_calc_An_GasEOut(An_DigNDF, animal_input['An_StatePhys'], diet_info, animal_input['DMI'])

    # Metabolizable Protein Requirements
    An_MPuse_g_Trg, An_MPm_g_Trg, Body_MPUse_g_Trg, Gest_MPUse_g_Trg, Mlk_MPUse_g_Trg, An_MPuse_kg_Trg = nd.calculate_MP_requirement(An_DEInp, An_DENPNCPIn, An_DigTPaIn, An_GasEOut, Frm_NPgain, diet_info.loc['Diet', 'Fd_NDFIn'], animal_input['DMI'], animal_input['An_BW'], animal_input['An_BW_mature'], animal_input['Trg_FrmGain'],
                                                                                                                 animal_input['Trg_RsrvGain'], animal_input['Trg_MilkProd'], animal_input['Trg_MilkTPp'], GrUter_BWgain, animal_input['An_StatePhys'], nd.coeff_dict)    

########################################
# Step 9: Performance Calculations
########################################
    if animal_input['An_StatePhys'] == 'Lactating Cow':
        # Predicted milk fat
        Mlk_Fat_g, An_LactDay_MlkPred = nd.calculate_Mlk_Fat_g(AA_values, diet_info.loc['Diet', 'Fd_FAIn'], diet_info.loc['Diet', 'Fd_DigC160In'], diet_info.loc['Diet', 'Fd_DigC183In'], animal_input['An_LactDay'], animal_input['DMI'])

        # Predicted milk yield
        Mlk_Prod_comp = nd.calculate_Mlk_Prod_comp(Mlk_NP_g, Mlk_Fat_g, An_DEIn, An_LactDay_MlkPred, animal_input['An_Parity_rl']) 
    
        # MP Allowable Milk
        Mlk_Prod_MPalow = nd.calculate_Mlk_Prod_MPalow(An_MPuse_g_Trg, Mlk_MPUse_g_Trg, An_MPIn, animal_input['Trg_MilkTPp'], nd.coeff_dict)

        # NE Allowable Milk
        Mlk_Prod_NEalow = nd.calculate_Mlk_Prod_NEalow(An_MEIn, An_MEgain, An_MEmUse, Gest_MEuse, Trg_NEmilk_Milk, nd.coeff_dict)
    else:
        Mlk_Fat_g = None
        Mlk_Prod_comp = None
        Mlk_Prod_MPalow = None
        Mlk_Prod_NEalow = None

########################################
# Step 10: Calculations Requiring Milk Production Values
########################################

    if animal_input['An_StatePhys'] == 'Lactating Cow':
        MlkNP_Milk = nd.temp_MlkNP_Milk(animal_input['An_StatePhys'], Mlk_NP_g, Mlk_Prod_comp, animal_input['Trg_MilkProd'])
    else:
        MlkNP_Milk = 0
    
    # Mineral Requirements
    mineral_requirements_df, An_DCADmeq = nd.mineral_requirements(animal_input['An_StatePhys'], animal_input['An_Parity_rl'], animal_input['An_Breed'], animal_input['DMI'], animal_input['An_BW_mature'], animal_input['An_BW'], animal_input['Trg_FrmGain'], animal_input['Trg_RsrvGain'], animal_input['An_GestDay'], GrUter_Wt, Mlk_NP_g, animal_input['Trg_MilkProd'], animal_input['Trg_MilkTPp'],
                                                                mineral_values.loc['Ca', 'Abs_mineralIn'], MlkNP_Milk, mineral_values.loc['P', 'Abs_mineralIn'], mineral_values.loc['P', 'Dt_mineralIn'], mineral_values.loc['Mg', 'Abs_mineralIn'], mineral_values.loc['Na', 'Abs_mineralIn'], mineral_values.loc['Cl', 'Abs_mineralIn'], mineral_values.loc['K', 'Abs_mineralIn'],
                                                                mineral_values.loc['S', 'Dt_mineralIn'], mineral_values.loc['Co', 'Abs_mineralIn'], mineral_values.loc['Cu', 'Abs_mineralIn'], mineral_values.loc['I', 'Dt_mineralIn'], mineral_values.loc['Fe', 'Abs_mineralIn'], mineral_values.loc['Mn', 'Abs_mineralIn'], mineral_values.loc['Se', 'Dt_mineralIn'], 
                                                                mineral_values.loc['Zn', 'Abs_mineralIn'], mineral_values.loc['K', 'Dt_macro'], mineral_values.loc['Na', 'Dt_macro'], mineral_values.loc['Cl', 'Dt_macro'], mineral_values.loc['S', 'Dt_macro'])

########################################
# Step 11: *Temporary* Add values of interest to list
########################################
    if animal_input['An_StatePhys'] == 'Lactating Cow':
        # Milk Fat %
        milk_fat = (Mlk_Fat_g / 1000) / Mlk_Prod_comp * 100
        # Milk Protein %
        milk_protein = (Mlk_NP_g / 1000) / Mlk_Prod_comp * 100
    else:
        milk_fat = None
        milk_protein = None

    model_results = {
    'milk_fat': milk_fat,
    'milk_protein': milk_protein,
    'Mlk_Prod_comp': Mlk_Prod_comp,
    'Mlk_Prod_MPalow': Mlk_Prod_MPalow,
    'Mlk_Prod_NEalow': Mlk_Prod_NEalow,
    'Trg_MEuse': Trg_MEuse,
    'An_MPuse_g_Trg': An_MPuse_g_Trg
}

    ####################
    # Testing Statements
    ####################
 
 
    return diet_info, animal_input, feed_data, equation_selection, AA_values, model_results, df_minerals, mineral_values, df_vitamins, mineral_requirements_df, An_DCADmeq

### RUN MODEL ###
diet_info, animal_input, feed_data, equation_selection, AA_values, model_results, df_minerals, mineral_values, df_vitamins, mineral_requirements_df, An_DCADmeq = run_dev_model()


164.4205405239588


In [3]:
# Vitamin Requirements

def vitamin_requirement(Trg_MilkProd, An_Parity_rl, An_BW, Dt_VitAIn, Dt_VitDIn, Dt_VitEIn):
    # Vit A, IU/d
    if Trg_MilkProd > 35:
        An_VitA_req = 110 * An_BW + 1000 * (Trg_MilkProd - 35)
    else:
        An_VitA_req = 110 * An_BW
    
    An_VitA_bal = Dt_VitAIn - An_VitA_req

    # Vit D, IU/d
    if Trg_MilkProd > 0:
        An_VitD_req = 110 * An_BW + 1000 * (Trg_MilkProd - 35)
    else:
        An_VitD_req = 110 * An_BW
    
    An_VitD_bal = Dt_VitDIn - An_VitD_req

    # Vit E, IU/d
    if Trg_MilkProd == 0 and An_Parity_rl >= 1:
        An_VitE_req = 2.0 * An_BW
    else: 
        An_VitE_req = 0.8 * An_BW

    if An_StatePhys == 'Calf':
        An_VitE_req = 2.0 * An_BW
    elif An_GestDay >= 259 and An_Preg == 1:
        An_VitE_req = 3.0 * An_BW
    
    An_VitE_req = An_VitE_req - Dt_PastIn * 50  #50 IU provided per kg of pasture DM

    if An_VitE_req < 0:
        An_VitE_req = 0
    
    An_VitE_bal = Dt_VitEIn - An_VitE_req

    return An_VitA_req, An_VitD_req, An_VitE_req

In [4]:
# Kept the new functions here temporarily in case of issue

# def read_csv_input(path_to_file):
    
#     animal_input = {}
#     equation_selection = {}
#     diet_info_data = {'Feedstuff': [], 'kg_user': []}

#     input_data = pd.read_csv(path_to_file)

#     for index, row in input_data.iterrows():
#         location = row['Location']
#         variable = row['Variable']
#         value = row['Value']
        
#         if location == 'equation_selection':
#             equation_selection[variable] = float(value) if value.replace('.', '', 1).isdigit() else value
#         elif location == 'animal_input':
#             animal_input[variable] = float(value) if value.replace('.', '', 1).isdigit() else value
#         elif location == 'diet_info':
#             diet_info_data['Feedstuff'].append(variable)
#             diet_info_data['kg_user'].append(value)
      
#     diet_info = pd.DataFrame(diet_info_data)
#     diet_info['kg_user'] = pd.to_numeric(diet_info['kg_user'], downcast="float")

#     return diet_info, animal_input, equation_selection


# def dry_cow_equations(DMIn_eqn, An_BW, An_PrePartWk, An_GestDay, An_GestLength, Dt_NDF, coeff_dict):
#     req_coeffs = ['Ka_LateGest_DMIn', 'Kc_LateGest_DMIn']
#     nd.check_coeffs_in_coeff_dict(coeff_dict, req_coeffs)

#     Dt_NDF_drylim = Dt_NDF
#     #constrain Dt_NDF to the range of 30 to 55% of DM
#     if Dt_NDF < 30:
#         Dt_NDF_drylim = 30
#     elif Dt_NDF > 55:
#         Dt_NDF_drylim = 55

#     #Late Gestation eqn. from Hayirli et al., 2003) for dry cows and heifers
#     if An_PrePartWk < -3:
#         An_PrePartWklim = -3
#     elif An_PrePartWk > 0:
#         An_PrePartWklim = 0
#     else:
#         An_PrePartWklim = An_PrePartWk
#     An_PrePartWkDurat = An_PrePartWklim * 2

#     Kb_LateGest_DMIn = -(0.365 - 0.0028 * Dt_NDF_drylim)
#     Dt_DMIn_BW_LateGest_i = coeff_dict['Ka_LateGest_DMIn'] + Kb_LateGest_DMIn * An_PrePartWklim + coeff_dict['Kc_LateGest_DMIn'] * An_PrePartWklim**2
#     Dt_DMIn_BW_LateGest_p = (coeff_dict['Ka_LateGest_DMIn'] * An_PrePartWkDurat + Kb_LateGest_DMIn / 2 * An_PrePartWkDurat**2 + coeff_dict['Kc_LateGest_DMIn'] / 3 * An_PrePartWkDurat**3) / An_PrePartWkDurat
    
#     Dt_DMIn_DryCow1_FarOff = An_BW * Dt_DMIn_BW_LateGest_i / 100
#     Dt_DMIn_DryCow1_Close = An_BW * Dt_DMIn_BW_LateGest_p / 100

#     if An_PrePartWk > An_PrePartWkDurat:
#         Dt_DMIn_DryCow1 = min(Dt_DMIn_DryCow1_FarOff, Dt_DMIn_DryCow1_Close)
#     else:
#         Dt_DMIn_DryCow1 = Dt_DMIn_DryCow1_FarOff

#     ## from Hayirli et al., 2003 JDS
#     if (An_GestDay - An_GestLength) < -21:
#         Dt_DMIn_DryCow_AdjGest = 0
#     else:
#         Dt_DMIn_DryCow_AdjGest = An_BW * (-0.756 * math.exp(0.154 * (An_GestDay - An_GestLength))) / 100
    
#     Dt_DMIn_DryCow2 = An_BW * 1.979 / 100 + Dt_DMIn_DryCow_AdjGest

#     if DMIn_eqn == 10:
#         return_DMI = Dt_DMIn_DryCow1
#     elif DMIn_eqn == 11:
#         return_DMI = Dt_DMIn_DryCow2

#     return return_DMI


# def heifer_growth(DMIn_eqn, Dt_NDF, An_BW, An_BW_mature, An_PrePartWk, coeff_dict):
# # Lines 317-379
#     req_coeffs = ['Ka_LateGest_DMIn', 'Kc_LateGest_DMIn']     
#     nd.check_coeffs_in_coeff_dict(coeff_dict, req_coeffs)

# #NRC 2020 Heifer Eqns. from the Transition Ch.
#     Dt_NDFdev_DMI = Dt_NDF - (23.11 + 0.07968 * An_BW - 0.00006252 * An_BW**2)      # Line 316
# #Animal factors only, eqn. 2-3 NRC
#     Dt_DMIn_Heif_NRCa = 0.022 * An_BW_mature * (1 - math.exp(-1.54 * An_BW / An_BW_mature))
# #Holstein, animal factors only
#     Dt_DMIn_Heif_H1 = 15.36 * (1 - math.exp(-0.0022 * An_BW))
# #Holstein x Jersey, animal factors only
#     Dt_DMIn_Heif_HJ1 = 12.91 * (1 - math.exp(-0.00295 * An_BW))
# #Anim & diet factors, eqn 2-4 NRC
#     Dt_DMIn_Heif_NRCad = (0.0226 * An_BW_mature * (1 - math.exp(-1.47 * An_BW / An_BW_mature))) - (0.082 * (Dt_NDF - (23.1 + 56 * An_BW / An_BW_mature) - 30.6 *(An_BW / An_BW_mature)**2))
# #Holstein, animal factors and NDF
#     Dt_DMIn_Heif_H2 = 15.79 * (1 - math.exp(-0.0021 * An_BW)) - (0.082 * Dt_NDFdev_DMI)
# #Holstein x Jersey, animal factors and NDF
#     Dt_DMIn_Heif_HJ2 = 13.48 * (1 - math.exp(-0.0027 * An_BW)) - (0.082 * Dt_NDFdev_DMI)

# #Late Gestation eqn. from Hayirli et al., 2003) for dry cows and heifers
#     if An_PrePartWk < -3:       #constrain to the interval 0 to -3.
#         An_PrePartWklim = -3
#     else:
#         An_PrePartWklim = An_PrePartWk

#     if An_PrePartWk > 0:
#         An_PrePartWklim = 0

# #the estimated length of time in the close-up pen
#     An_PrePartWkDurat = An_PrePartWklim * 2  

#     Dt_NDF_drylim = Dt_NDF

#     if Dt_NDF < 30:    #constrain Dt_NDF to the range of 30 to 55% of DM
#         Dt_NDF_drylim = 30
#     if Dt_NDF > 55:
#         Dt_NDF_drylim = 55

#     Kb_LateGest_DMIn = -(0.365 - 0.0028 * Dt_NDF_drylim)

# #These 2 values are called by other functions, must be calculated everytime
# #Late gestation individual animal prediction, % of BW.  Use to assess for a specific day for a given animal
#     Dt_DMIn_BW_LateGest_i = coeff_dict['Ka_LateGest_DMIn'] + Kb_LateGest_DMIn * An_PrePartWklim + coeff_dict['Kc_LateGest_DMIn'] * An_PrePartWklim**2
# #Late gestation Group/Pen mean DMI/BW for an interval of 0 to PrePart_WkDurat.  Assumes pen steady state and PrePart_wk = pen mean
#     Dt_DMIn_BW_LateGest_p = (coeff_dict['Ka_LateGest_DMIn'] * An_PrePartWkDurat + Kb_LateGest_DMIn / 2 * An_PrePartWkDurat**2 + coeff_dict['Kc_LateGest_DMIn'] / 3 * An_PrePartWkDurat**3) / An_PrePartWkDurat

# #Individual intake for the specified day prepart or the pen mean intake for the interval, 0 to PrePart_WkDurat
#     Dt_DMIn_Heif_LateGestInd = 0.88 * An_BW * Dt_DMIn_BW_LateGest_i / 100 #Individual animal
#     Dt_DMIn_Heif_LateGestPen = 0.88 * An_BW * Dt_DMIn_BW_LateGest_p / 100 #Pen mean

# #Switch to the group transition eqn. when less than An_PrePartWkDurat and predicted transition DMI is less than far off DMI
# #These equations generally are discontinuous at -3 weeks, as Dt_DMIn_xxx is greater than Dt_DMIn_xxx_LateGest at -3.  
# #Should calculate the decline using the far off DMIn as a reference point, or calculate the discontinuity at the
# #point of transition and adjust the close-up DMIn by addition of the gap.

# #NRC 2020 pen intakes
#     if An_PrePartWk > An_PrePartWkDurat:
#         Dt_DMIn_Heif_NRCap = min(Dt_DMIn_Heif_NRCa, Dt_DMIn_Heif_LateGestPen)
#         Dt_DMIn_Heif_NRCadp = min(Dt_DMIn_Heif_NRCad, Dt_DMIn_Heif_LateGestPen)
#         Dt_DMIn_Heif_H1p =  min(Dt_DMIn_Heif_H1, Dt_DMIn_Heif_LateGestPen)
#         Dt_DMIn_Heif_HJ1p = min(Dt_DMIn_Heif_HJ1, Dt_DMIn_Heif_LateGestPen)
#         Dt_DMIn_Heif_H2p = min(Dt_DMIn_Heif_H2, Dt_DMIn_Heif_LateGestPen)
#         Dt_DMIn_Heif_HJ2p = min(Dt_DMIn_Heif_HJ2, Dt_DMIn_Heif_LateGestPen)
#     else:
#         Dt_DMIn_Heif_NRCap = Dt_DMIn_Heif_NRCa
#         Dt_DMIn_Heif_NRCadp = Dt_DMIn_Heif_NRCad
#         Dt_DMIn_Heif_H1p = Dt_DMIn_Heif_H1
#         Dt_DMIn_Heif_HJ1p = Dt_DMIn_Heif_HJ1
#         Dt_DMIn_Heif_H2p = Dt_DMIn_Heif_H2
#         Dt_DMIn_Heif_HJ2p = Dt_DMIn_Heif_HJ2

# #NRC 2020 individual animal intakes
#     if An_PrePartWk > An_PrePartWkDurat:
#         Dt_DMIn_Heif_NRCai = min(Dt_DMIn_Heif_NRCa, Dt_DMIn_Heif_LateGestInd)
#         Dt_DMIn_Heif_NRCadi =  min(Dt_DMIn_Heif_NRCad, Dt_DMIn_Heif_LateGestInd)
#         Dt_DMIn_Heif_H1i = min(Dt_DMIn_Heif_H1, Dt_DMIn_Heif_LateGestInd)
#         Dt_DMIn_Heif_HJ1i = min(Dt_DMIn_Heif_HJ1, Dt_DMIn_Heif_LateGestInd)
#         Dt_DMIn_Heif_H2i = min(Dt_DMIn_Heif_H2, Dt_DMIn_Heif_LateGestInd)
#         Dt_DMIn_Heif_HJ2i = min(Dt_DMIn_Heif_HJ2,Dt_DMIn_Heif_LateGestInd)
#     else:
#         Dt_DMIn_Heif_NRCai = Dt_DMIn_Heif_NRCa
#         Dt_DMIn_Heif_NRCadi = Dt_DMIn_Heif_NRCad
#         Dt_DMIn_Heif_H1i = Dt_DMIn_Heif_H1
#         Dt_DMIn_Heif_HJ1i = Dt_DMIn_Heif_HJ1
#         Dt_DMIn_Heif_H2i = Dt_DMIn_Heif_H2
#         Dt_DMIn_Heif_HJ2i = Dt_DMIn_Heif_HJ2
    
#     # Select the user specified DMI to return
#     # This selection could be moved higher up in the function
#     if DMIn_eqn == 2:
#         return_DMI = Dt_DMIn_Heif_NRCai
#     elif DMIn_eqn == 3:
#         return_DMI = Dt_DMIn_Heif_NRCadi
#     elif DMIn_eqn == 4:
#         return_DMI = Dt_DMIn_Heif_H1i
#     elif DMIn_eqn == 5:
#         return_DMI = Dt_DMIn_Heif_H2i
#     elif DMIn_eqn == 6:
#         return_DMI = Dt_DMIn_Heif_HJ1i
#     elif DMIn_eqn == 7:
#         return_DMI = Dt_DMIn_Heif_HJ2i
#     elif DMIn_eqn == 12:
#         return_DMI = Dt_DMIn_Heif_NRCap
#     elif DMIn_eqn == 13:
#         return_DMI = Dt_DMIn_Heif_NRCadp
#     elif DMIn_eqn == 14:
#         return_DMI = Dt_DMIn_Heif_H1p
#     elif DMIn_eqn == 15:
#         return_DMI = Dt_DMIn_Heif_H2p
#     elif DMIn_eqn == 16:
#         return_DMI = Dt_DMIn_Heif_HJ1p
#     elif DMIn_eqn == 17:
#         return_DMI = Dt_DMIn_Heif_HJ2p

#     return return_DMI


# def mineral_intakes(An_StatePhys, feed_data, diet_info, Dt_DMIn_ClfLiq=0):
#     # Currently calf calculations have not been incorporated into the model, Calf liquid feed DMI (Dt_DMIn_ClfLiq) is therefore set to 0
#     # Once calf calculations are added a real value can be passed here

#     # Lists of macro and micro mineral abreviations
#     macro_minerals = ['Ca', 'P', 'Pinorg', 'Porg', 'Na', 'Mg', 'K', 'Cl', 'S']
#     abs_macro_minerals = ['Ca', 'P', 'Mg', 'Na', 'K', 'Cl']
#     micro_minerals = ['Co', 'Cr', 'Cu', 'Fe', 'I', 'Mn', 'Mo', 'Se', 'Zn']
#     abs_micro_minerals = ['Co', 'Cu', 'Fe', 'Mn', 'Zn']

#     mineral_values = pd.DataFrame(index=[macro_minerals + micro_minerals])

#     ####################
#     # Get Diet Data
#     ####################
#     feed_columns = ['Fd_Category', 'Fd_Ca', 'Fd_P', 'Fd_Pinorg_P', 'Fd_Porg_P', 'Fd_Na', 'Fd_Mg', 'Fd_K', 'Fd_Cl', 'Fd_S',
#                     'Fd_Co', 'Fd_Cr', 'Fd_Cu', 'Fd_Fe', 'Fd_I', 'Fd_Mn', 'Fd_Mo', 'Fd_Se', 'Fd_Zn', 'Fd_acCa', 'Fd_acPtot',
#                     'Fd_acMg', 'Fd_acNa', 'Fd_acK', 'Fd_acCl', 'Fd_acCu', 'Fd_acFe', 'Fd_acMn', 'Fd_acZn']
#     df_minerals = pd.DataFrame(feed_data[feed_columns])

#     # Get DMI for each feed from diet_info using the index of df_min to match the values
#     for feed in df_minerals.index:
#         if feed in diet_info['Feedstuff'].values:
#             df_minerals.at[feed, 'Fd_DMIn'] = diet_info[diet_info['Feedstuff'] == feed]['kg_user'].values[0]

#     # Save DMI as variable
#     Dt_DMIn = df_minerals['Fd_DMIn'].sum()  

#     # Adjust values based on An_StatePhys and Fd_Category
#     df_minerals.loc[df_minerals['Fd_Category'] != 'Vitamin/Mineral', 'Fd_MgIn_min'] = 0
#     df_minerals.loc[df_minerals['Fd_Category'] == 'Vitamin/Mineral', 'Fd_MgIn_min'] = df_minerals['Fd_DMIn'] * df_minerals['Fd_Mg'] / 100 * 1000

#     # Adjust absorption coefficients for calves
#     if An_StatePhys == 'Calf':
#         df_minerals['Fd_acNa'] = 1
#         df_minerals['Fd_acK'] = 1
#         df_minerals['Fd_acCu'] = 1.0    # Line 1861
#         df_minerals['Fd_acFe'] = 1.0    # Line 1863
#         df_minerals['Fd_acMn'] = 1.0    # Line 1865
#         df_minerals['Fd_acZn'] = 1.0    # Line 1867
    
#     if An_StatePhys == 'Calf' and Dt_DMIn_ClfLiq > 0:
#         df_minerals['Fd_acCa'] = 1  # Line 1839
#         df_minerals['Fd_acPtot'] = 1 # Line 1842
#         df_minerals['Fd_acMg'] = 1  # Line 1844
#         df_minerals['Fd_acCl'] = 1  # Line 1848
#     elif An_StatePhys == 'Calf' and Dt_DMIn_ClfLiq == 0:
#         df_minerals['Fd_acCa'] = 0.60   # Line 1840
#         df_minerals['Fd_acPtot'] = 0.75 # Line 1843
#         df_minerals['Fd_acMg'] = 0.26   # Line 1845
#         df_minerals['Fd_acCl'] = 0.92   # Line 1849
#         df_minerals['Fd_acCu'] = 0.10   # Line 1862
#         df_minerals['Fd_acFe'] = 0.10   # Line 1864
#         df_minerals['Fd_acMn'] = 0.005  # Line 1866
#         df_minerals['Fd_acZn'] = 0.20   # Line 1868
    
#     df_minerals.loc[df_minerals['Fd_Category'] != 'Vitamin/Mineral', 'Fd_acPtot'] = df_minerals['Fd_Pinorg_P'] * 0.0084 + df_minerals['Fd_Porg_P'] * 0.0068 # Line 1841
    
#     if An_StatePhys == 'Calf':  # Line 1860
#         df_minerals['Fd_acCo'] = 0
#     else:
#         df_minerals['Fd_acCo'] = 1.0


#     ####################
#     # Macromineral Supply and Absorption
#     ####################
#     for macro in macro_minerals:
#         # Macromineral intakes for each ingredient, g/d
#         if 'Fd_{}'.format(macro) in df_minerals.columns:
#             df_minerals['Fd_{}In'.format(macro)] = df_minerals['Fd_DMIn'] * df_minerals['Fd_{}'.format(macro)] / 100 * 1000
#     # Pinorg and Porg have a different first step, require Fd_PIn from above step
#     df_minerals['Fd_PinorgIn'] = df_minerals['Fd_PIn'] * df_minerals['Fd_Pinorg_P'] / 100   #??Check Bill's text
#     df_minerals['Fd_PorgIn'] = df_minerals['Fd_PIn'] * df_minerals['Fd_Porg_P'] / 100  #Depracated by Bill.  Reduced to inorganic and organic.     

#     for macro in macro_minerals:
#         # Total diet macromineral intakes
#         mineral_values.loc[macro, 'Dt_mineralIn'] = df_minerals['Fd_{}In'.format(macro)].sum()  # Lines 762-771
#         # Macro minerals, % of DM
#         mineral_values.loc[macro, 'Dt_macro'] = mineral_values.loc[macro, 'Dt_mineralIn'] / Dt_DMIn / 1000 * 100  
    
#     for macro in abs_macro_minerals:
#         if macro == 'P':
#             df_minerals['Fd_absPIn'] = df_minerals['Fd_PIn'] * df_minerals['Fd_acPtot'] # Line 1852
#         elif macro == 'Mg':
#             df_minerals['Fd_absMgIn_base'] = df_minerals['Fd_MgIn'] * df_minerals['Fd_acMg'] # Line 1853
#         else:
#             df_minerals['Fd_abs{}In'.format(macro)] = df_minerals['Fd_{}In'.format(macro)] * df_minerals['Fd_ac{}'.format(macro)] # Line 1851, 1854-1856

#     for macro in abs_macro_minerals:
#         if macro == 'Mg':
#             if An_StatePhys == 'Calf':
#                 Dt_acMg = 1
#             else:
#                 # Dt_acMg = (44.1 - 5.42 * math.log(mineral_values.loc['K', 'Dt_macro'] * 10) - 0.08 * df_minerals['Fd_MgIn_min'].sum() / df_minerals['Fd_MgIn'].sum() * 100) / 100
#                 Dt_acMg = (44.1 - 5.42 * math.log(float(mineral_values.loc['K', 'Dt_macro']) * 10) - 0.08 * df_minerals['Fd_MgIn_min'].sum() / df_minerals['Fd_MgIn'].sum() * 100) / 100
#             mineral_values.loc[macro, 'Abs_mineralIn'] = Dt_acMg * df_minerals['Fd_MgIn_min'].sum()
#         else:
#             mineral_values.loc[macro, 'Abs_mineralIn'] = df_minerals['Fd_abs{}In'.format(macro)].sum()


#     ####################
#     # Micromineral Supply and Absorption
#     ####################
#     for micro in micro_minerals:
#     #Micro Mineral Intakes from each ingredient, mg/d
#         df_minerals['Fd_{}In'.format(micro)] = df_minerals['Fd_DMIn'] * df_minerals['Fd_{}'.format(micro)] # Lines 741-749
#     #Total Dietary Micro Mineral Intakes, mg/d
#         mineral_values.loc[micro, 'Dt_mineralIn'] = df_minerals['Fd_{}In'.format(micro)].sum()  # Lines 774-782
#     #Total Dietary Micro Mineral Intakes, mg/kg
#         mineral_values.loc[micro, 'Dt_micro'] = mineral_values.loc[micro, 'Dt_mineralIn'] / Dt_DMIn     # Lines 807-815

#     for micro in abs_micro_minerals:
#         df_minerals['Fd_abs{}In'.format(micro)] = df_minerals['Fd_{}In'.format(micro)] * df_minerals['Fd_ac{}'.format(micro)]  # Lines 1870-1874
#         mineral_values.loc[micro, 'Abs_mineralIn'] = df_minerals['Fd_abs{}In'.format(micro)].sum()  # Line 1884-1888

#     # Drop the columns from feed_data
#     df_minerals = df_minerals.drop(columns=feed_columns)

#     return df_minerals, mineral_values


# def vitamin_supply(feed_data, diet_info):
#     vitamins = ['VitA', 'VitD', 'VitE', 'Choline', 'Biotin', 'Niacin', 'B_Carotene']
#     vitamin_values = pd.DataFrame(index=vitamins)

#     ####################
#     # Get Diet Data
#     ####################
#     feed_columns = ['Fd_VitA', 'Fd_VitD', 'Fd_VitE', 'Fd_Choline', 'Fd_Biotin', 'Fd_Niacin', 'Fd_B_Carotene']
#     df_vitamins = pd.DataFrame(feed_data[feed_columns])

#     # Get DMI for each feed from diet_info using the index of df_min to match the values
#     for feed in df_vitamins.index:
#         if feed in diet_info['Feedstuff'].values:
#             df_vitamins.at[feed, 'Fd_DMIn'] = diet_info[diet_info['Feedstuff'] == feed]['kg_user'].values[0]

#     ####################
#     # Vitamin Supply
#     ####################
#     for vitamin in vitamins:
#         df_vitamins['Fd_{}In'.format(vitamin)] = df_vitamins['Fd_DMIn'] * df_vitamins['Fd_{}'.format(vitamin)] # Lines 752-759

#         vitamin_values.loc[vitamin, 'Dt_vitaminIn'] = df_vitamins['Fd_{}In'.format(vitamin)].sum()  # Lines 785-791

#         vitamin_values.loc[vitamin, 'Dt_{}'.format(vitamin)] = vitamin_values.loc[vitamin, 'Dt_vitaminIn'] / df_vitamins['Fd_DMIn'].sum()   # Lines 818-825

#     return df_vitamins


# def mineral_requirements(An_StatePhys, An_Parity_rl, An_Breed, An_DMIn, An_BW_mature, An_BW, Trg_FrmGain, Trg_RsrvGain, An_GestDay, GrUter_Wt, Mlk_NP_g, Trg_MilkProd, Trg_MilkTPp, Abs_CaIn, MlkNP_Milk, Abs_PIn, Dt_PIn, Abs_MgIn, Abs_NaIn,
#                          Abs_ClIn, Abs_KIn, Dt_SIn, Abs_CoIn, Abs_CuIn, Dt_IIn, Abs_FeIn, Abs_MnIn, Dt_SeIn, Abs_ZnIn, Dt_K, Dt_Na, Dt_Cl, Dt_S):
    
#     # Calculations that should be moved in the future
#     Frm_Gain = Trg_FrmGain
#     Rsrv_Gain = Trg_RsrvGain
#     Rsrv_Gain_empty = Rsrv_Gain 
#     Body_Gain = Frm_Gain + Rsrv_Gain    # Line 2436

#     if An_StatePhys in ["Dry Cow", "Lactating Cow"] and An_Parity_rl > 0:
#         An_GutFill_BW = 0.18
#     else:
#         An_GutFill_BW = 0.06

#     Frm_Gain_empty = Frm_Gain * (1 - An_GutFill_BW)	    #Assume the same gut fill for frame gain
#     Body_Gain_empty = Frm_Gain_empty + Rsrv_Gain_empty


#     An_BWnp = An_BW - GrUter_Wt  #Non-pregnant BW
#     An_GutFill_Wt = An_GutFill_BW * An_BWnp
#     An_BW_empty = An_BW - An_GutFill_Wt 	

#     ############
#     # Calcium g/d, Lines 2962-2973
#     ############
#     if An_Breed == 'Jersey':    
#         Ca_Mlk = 1.17
#     else:
#         Ca_Mlk = 1.03   
    
#     Fe_Ca_m = 0.9 * An_DMIn     #Maintenance
#     An_Ca_g = (9.83 * An_BW_mature**0.22 * An_BW**-.22) * Body_Gain     #Growth
#     An_Ca_y = (0.0245 * math.exp((0.05581 - 0.00007 * An_GestDay) * An_GestDay)- 0.0245 * math.exp((0.05581 - 0.00007 * (An_GestDay - 1)) * (An_GestDay - 1))) * An_BW / 715    #Gestation 
#     if math.isnan(Mlk_NP_g) is True:    #Lactation
#         An_Ca_l = Ca_Mlk * Trg_MilkProd
#     else: 
#         An_Ca_l = (0.295 + 0.239 * Trg_MilkTPp) * Trg_MilkProd
#     if math.isnan(An_Ca_l) is True:
#         An_Ca_l = 0
#     An_Ca_Clf = (0.0127 * An_BW_empty + (14.4 * (An_BW_empty**-0.139) * Body_Gain_empty)) / 0.73
#     if An_StatePhys == 'Calf' and Dt_DMIn_ClfLiq > 0:
#         An_Ca_req = An_Ca_Clf
#     else:
#         An_Ca_req = Fe_Ca_m + An_Ca_g + An_Ca_y + An_Ca_l
    
#     An_Ca_bal = Abs_CaIn - An_Ca_req
#     An_Ca_prod = An_Ca_y + An_Ca_l + An_Ca_g

#     ############
#     # Phosphorus g/d, Lines 2976-2991
#     ############
#     Ur_P_m = 0.0006 * An_BW
#     if An_Parity_rl == 0:
#         Fe_P_m = 0.8 * An_DMIn
#     else:
#         Fe_P_m = 1.0 * An_DMIn
#     An_P_m = Ur_P_m + Fe_P_m
#     An_P_g = (1.2 + (4.635 * An_BW_mature**.22 * An_BW**-0.22)) * Body_Gain
#     An_P_y = (0.02743 * math.exp((0.05527 - 0.000075 * An_GestDay) * An_GestDay) - 0.02743 * math.exp((0.05527 - 0.000075 * (An_GestDay - 1)) * (An_GestDay - 1))) * An_BW / 715
#     if math.isnan(Trg_MilkProd) is True:
#         An_P_l = 0      #If MTP not known then 0.9*Milk
#     else:
#         An_P_l = (0.48 + 0.13 * MlkNP_Milk * 100) * Trg_MilkProd
#     An_P_Clf = (0.0118 * An_BW_empty + (5.85 * (An_BW_empty**-0.027) * Body_Gain_empty)) / 0.65
#     if An_StatePhys == 'Calf' and Dt_DMIn_ClfLiq > 0:
#         An_P_req = An_P_Clf
#     else:
#         An_P_req = An_P_m + An_P_g + An_P_y + An_P_l
#     An_P_bal = Abs_PIn - An_P_req   #Infused P not currently considered as an input, but should be
#     Fe_P_g = Dt_PIn - An_P_l - An_P_y - An_P_g - Ur_P_m
#       #urinary losses will be underestimated at very high dietary P. Ordinarily 99% by feces.
#     An_P_prod = An_P_y + An_P_l + An_P_g

#     ############
#     # Magnesium g/d, Lines 2994-3004
#     ############
#     An_Mg_Clf = (0.0035 * An_BW_empty + (0.60 * (An_BW_empty**-0.036) * Body_Gain_empty)) / 0.30
#     Ur_Mg_m = 0.0007 * An_BW
#     Fe_Mg_m = 0.3 * An_DMIn
#     An_Mg_m = Ur_Mg_m + Fe_Mg_m
#     An_Mg_g = 0.45 * Body_Gain
#     if An_GestDay > 190:
#         An_Mg_y = 0.3 * (An_BW / 715)
#     else:
#         An_Mg_y = 0

#     if math.isnan(Trg_MilkProd) is True:
#         An_Mg_l = 0
#     else:
#         An_Mg_l = 0.11 * Trg_MilkProd

#     if An_StatePhys == 'Calf' and Dt_DMIn_ClfLiq > 0:
#         An_Mg_req = An_Mg_Clf
#     else:
#         An_Mg_req = An_Mg_m + An_Mg_g + An_Mg_y + An_Mg_l

#     An_Mg_bal = Abs_MgIn - An_Mg_req
#     An_Mg_prod = An_Mg_y + An_Mg_l + An_Mg_g

#     ############
#     # Sodium g/d, Lines 3007-3015
#     ############
#     An_Na_Clf = (0.00637 * An_BW_empty + (1.508 * (An_BW_empty**-0.045) * Body_Gain_empty)) / 0.24
#     Fe_Na_m = 1.45 * An_DMIn
#     An_Na_g = 1.4 * Body_Gain
#     if An_GestDay > 190:
#         An_Na_y = 1.4 * An_BW / 715
#     else:
#         An_Na_y = 0

#     if math.isnan(Trg_MilkProd) is True:
#         An_Na_l = 0
#     else:
#         An_Na_l = 0.4 * Trg_MilkProd

#     if An_StatePhys == 'Calf' and Dt_DMIn_ClfLiq > 0:
#         An_Na_req = An_Na_Clf
#     else:
#         An_Na_req = Fe_Na_m + An_Na_g + An_Na_y + An_Na_l

#     An_Na_bal = Abs_NaIn - An_Na_req
#     An_Na_prod = An_Na_y + An_Na_l + An_Na_g

#     ############
#     # Chlorine g/d, Lines 3017-3025
#     ############
#     An_Cl_Clf = 0.8 * (0.00637 * An_BW_empty + (1.508 * (An_BW_empty**-0.045) * Body_Gain_empty)) / 0.24
#     Fe_Cl_m = 1.11 * An_DMIn
#     An_Cl_g = 1.0 * Body_Gain
#     if An_GestDay > 190:
#         An_Cl_y = 1.0 * An_BW / 715       
#     else:
#        An_Cl_y = 0

#     if math.isnan(Trg_MilkProd) is True:
#         An_Cl_l = 0
#     else:
#         An_Cl_l = 1.0 * Trg_MilkProd

#     if An_StatePhys == 'Calf' and Dt_DMIn_ClfLiq > 0:
#         An_Cl_req = An_Cl_Clf
#     else:
#         An_Cl_req = Fe_Cl_m + An_Cl_g + An_Cl_y + An_Cl_l

#     An_Cl_bal = Abs_ClIn - An_Cl_req
#     An_Cl_prod = An_Cl_y + An_Cl_l + An_Cl_g

#     ############
#     # Potassium g/d, Lines 3027-3037
#     ############
#     An_K_Clf = (0.0203 * An_BW_empty + (1.14 * (An_BW_empty**-0.048) * Body_Gain_empty)) / 0.13
#     if Trg_MilkProd > 0:
#         Ur_K_m = 0.2 * An_BW
#     else:
#         Ur_K_m = 0.07 * An_BW

#     Fe_K_m = 2.5 * An_DMIn
#     An_K_m = Ur_K_m + Fe_K_m
#     An_K_g = 2.5 * Body_Gain

#     if An_GestDay > 190:
#         An_K_y = 1.03 * An_BW / 715
#     else:
#         An_K_y = 0

#     if math.isnan(Trg_MilkProd) is True:
#         An_K_l = 0
#     else:
#         An_K_l = 1.5 * Trg_MilkProd

#     if An_StatePhys == 'Calf' and Dt_DMIn_ClfLiq > 0:
#         An_K_req = An_K_Clf
#     else: 
#         An_K_req = An_K_m + An_K_g + An_K_y + An_K_l

#     An_K_bal = Abs_KIn - An_K_req
#     An_K_prod = An_K_y + An_K_l + An_K_g

#     ############
#     # Sulphur g/d, Lines 3039-3040
#     ############
#     An_S_req = 2 * An_DMIn
#     An_S_bal = Dt_SIn - An_S_req

#     ############
#     # Cobalt mg/d, Lines 3045-3046
#     ############
#     An_Co_req = 0.2 * An_DMIn  #Based on dietary intake assuming no absorption for a calf
#     An_Co_bal = Abs_CoIn - An_Co_req #calf absorption set to 0 and other StatePhys to 1 above

#     ############
#     # Copper mg/d, Lines 3049-3057
#     ############
#     An_Cu_Clf = (0.0145 * An_BW + 2.5 * Body_Gain_empty) / 0.5
#     An_Cu_m = 0.0145 * An_BW
#     An_Cu_g = 2.0 * Body_Gain
#     if An_GestDay < 90:
#         An_Cu_y = 0
#     elif An_GestDay > 190:
#         An_Cu_y = 0.0023 * An_BW
#     else:
#         An_Cu_y = 0.0003 * An_BW

#     if math.isnan(Trg_MilkProd) is True:
#         An_Cu_l = 0
#     else:
#         An_Cu_l = 0.04 * Trg_MilkProd

#     if An_StatePhys == 'Calf' and Dt_DMIn_ClfLiq > 0:
#         An_Cu_req = An_Cu_Clf
#     else:
#         An_Cu_req = An_Cu_m + An_Cu_g + An_Cu_y + An_Cu_l

#     An_Cu_bal = Abs_CuIn - An_Cu_req
#     An_Cu_prod = An_Cu_y + An_Cu_l + An_Cu_g

#     ############
#     # Iodine mg/d, Lines 3059-3060
#     ############
#     if An_StatePhys == 'Calf':
#         An_I_req = 0.8 * An_DMIn
#     else:
#         An_I_req = 0.216 * An_BW**0.528 + 0.1 * Trg_MilkProd
#     An_I_bal = Dt_IIn - An_I_req

#     ############
#     # Iron mg/d, Lines 3063-3070
#     ############
#     An_Fe_Clf = 34 * Body_Gain / 0.25
#     An_Fe_g = 34 * Body_Gain

#     if An_GestDay > 190:
#         An_Fe_y = 0.025 * An_BW
#     else:
#         An_Fe_y = 0

#     if math.isnan(Trg_MilkProd) is True:
#        An_Fe_l = 0
#     else:
#         An_Fe_l = 1.0 * Trg_MilkProd

#     if An_StatePhys == 'Calf' and Dt_DMIn_ClfLiq > 0:
#         An_Fe_req = An_Fe_Clf
#     else: 
#         An_Fe_req = An_Fe_g + An_Fe_y + An_Fe_l

#     An_Fe_bal = Abs_FeIn - An_Fe_req
#     An_Fe_prod = An_Fe_y + An_Fe_l + An_Fe_g

#     ############
#     # Manganese mg/d, Lines 3072-3080
#     ############
#     An_Mn_Clf = (0.0026 * An_BW + 0.7 * Body_Gain) / 0.01
#     An_Mn_m = 0.0026 * An_BW
#     An_Mn_g = 0.7 * Body_Gain

#     if An_GestDay > 190:
#         An_Mn_y = 0.00042 * An_BW
#     else:
#         An_Mn_y = 0

#     if math.isnan(Trg_MilkProd) is True:
#        An_Mn_l = 0
#     else:
#         An_Mn_l = 0.03 * Trg_MilkProd

#     if An_StatePhys == 'Calf' and Dt_DMIn_ClfLiq > 0:
#         An_Mn_req = An_Mn_Clf
#     else: 
#         An_Mn_req = An_Mn_m + An_Mn_g + An_Mn_y + An_Mn_l

#     An_Mn_bal = Abs_MnIn - An_Mn_req
#     An_Mn_prod = An_Mn_y + An_Mn_l + An_Mn_g

#     ############
#     # Selenium mg/d, Lines 3082-3083
#     ############
#     An_Se_req = 0.3 * An_DMIn
#     An_Se_bal = Dt_SeIn - An_Se_req

#     ############
#     # Zinc mg/d, Lines 3086-3094
#     ############
#     An_Zn_Clf = (2.0 * An_DMIn + 24 * Body_Gain) / 0.25
#     An_Zn_m = 5.0 * An_DMIn
#     An_Zn_g = 24 * Body_Gain

#     if An_GestDay > 190:
#         An_Zn_y = 0.017 * An_BW
#     else:
#         An_Zn_y = 0

#     if math.isnan(Trg_MilkProd) is True:
#        An_Zn_l = 0
#     else:
#         An_Zn_l = 4.0 * Trg_MilkProd

#     if An_StatePhys == 'Calf' and Dt_DMIn_ClfLiq > 0:
#         An_Zn_req = An_Zn_Clf
#     else: 
#         An_Zn_req = An_Zn_m + An_Zn_g + An_Zn_y + An_Zn_l

#     An_Zn_bal = Abs_ZnIn - An_Zn_req
#     An_Zn_prod = An_Zn_y + An_Zn_l + An_Zn_g
    
#     ############
#     # DCAD meg/kg, Line 3095
#     ############
#     An_DCADmeq = (Dt_K.values[0] / 0.039 + Dt_Na.values[0] / 0.023 - Dt_Cl.values[0] / 0.0355 - Dt_S.values[0] / 0.016) * 10

#     variable_names = ['An_Ca_req', 'An_P_req', 'An_Mg_req', 'An_Na_req', 'An_Cl_req', 'An_K_req', 'An_S_req', 'An_Co_req', 'An_Cu_req', 'An_I_req', 'An_Fe_req', 'An_Mn_req', 'An_Se_req', 'An_Zn_req']
#     variable_values = [An_Ca_req, An_P_req, An_Mg_req, An_Na_req, An_Cl_req, An_K_req, An_S_req, An_Co_req, An_Cu_req, An_I_req, An_Fe_req, An_Mn_req, An_Se_req, An_Zn_req]
#     mineral_requirements = {variable: number for variable, number in zip(variable_names, variable_values)}

#     return mineral_requirements, An_DCADmeq


# def temp_MlkNP_Milk(An_StatePhys, Mlk_NP_g, Mlk_Prod_comp, Trg_MilkProd):

#     if An_StatePhys == "Lactating Cow":
#         Mlk_Prod = Mlk_Prod_comp
#     else:
#         Mlk_Prod = Trg_MilkProd

#     MlkNP_Milk = Mlk_NP_g / 1000 / Mlk_Prod	#Milk true protein, g/g
#     if An_StatePhys != "Lactating Cow":
#         MlkNP_Milk = 0

#     return MlkNP_Milk


# def NDF_precalculation(diet_info, feed_data):
#     # Create df to perform calculations with
#     df_NDF = pd.DataFrame()
#     # Copy feed names from diet_info
#     df_NDF['Feedstuff'] = diet_info['Feedstuff'].copy()
#     df_NDF['kg_user'] = diet_info['kg_user'].copy()
#     # Drop the diet row from this df if it exists, this will prevent errors if a user called this function after running get_nutrient_intakes
#     df_NDF = df_NDF[df_NDF['Feedstuff'] != 'Diet']
#     # Get a % intake based on user entered kg/d
#     df_NDF['user_%_intake'] = df_NDF['kg_user'] / df_NDF['kg_user'].sum()
#     # Get NDF% of each feed as a decimal
#     df_NDF['Fd_NDF'] = df_NDF['Feedstuff'].map(feed_data['Fd_NDF']) / 100
#     # Calculate % NDF of diet
#     Dt_NDF = (df_NDF['Fd_NDF'] * df_NDF['user_%_intake']).sum()
#     # Cleanup
#     del(df_NDF)
   
#     return Dt_NDF


# def temp_calc_An_DigTPaIn(Fe_CP, diet_info):
#     # Dt_TPIn = sum(Fd_TPIn)
#     An_TPIn = diet_info.loc['Diet', 'Fd_TPIn']

#     An_DigTPaIn = An_TPIn  - Fe_CP #- InfArt_CPIn, no infusions in the model at this point

#     return An_DigTPaIn


# def temp_calc_An_GasEOut(An_DigNDF, An_StatePhys, diet_info, Dt_DMIn):
#     An_GasEOut_Dry = 0.69 + 0.053 * diet_info.loc['Diet', 'Fd_GEIn'] - 0.07 * diet_info.loc['Diet', 'Fd_FAIn'] / Dt_DMIn * 100 

#     An_GasEOut_Lact = 0.294 * Dt_DMIn - 0.347 * diet_info.loc['Diet', 'Fd_FAIn'] / Dt_DMIn * 100 + 0.0409 * An_DigNDF

#     An_GasEOut_Heif = -0.038 + 0.051 * diet_info.loc['Diet', 'Fd_GEIn'] + 0.0091 * diet_info.loc['Diet', 'Fd_NDF_%_diet'] * 100

#     if An_StatePhys == 'Dry Cow':
#         An_GasEOut = An_GasEOut_Dry
#     # elif An_StatePhys == 'Calf':
#     #     An_GasEOut = An_GasEOut_Clf
#     elif An_StatePhys == 'Lactating Cow':
#         An_GasEOut = An_GasEOut_Lact
#     elif An_StatePhys == 'Heifer':
#         An_GasEOut = An_GasEOut_Heif

#     return An_GasEOut

